<a href="https://colab.research.google.com/github/Edark94/Computer-Vision-Project/blob/master/ComputerVisionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Hello world!')


Hello world!
